# Curious Behavior Implementation

**Step 1: Create display widgets**

In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

ModuleNotFoundError: No module named 'ipywidgets'

**Step 2: Initialize camera and implement curious behavior**

The robot will exhibit curious behavior through three states:
- **Scanning**: Rotating to search for objects when none detected
- **Approaching**: Moving toward detected objects
- **Investigating**: Circling around objects at close range

After investigation, the robot returns to scanning mode to explore more.

In [ ]:
import numpy as np
import pyzed.sl as sl
import threading
import motors

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera():
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        # Flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)
        
        # Curiosity state variables
        self.curiosity_state = 'scanning'
        self.investigation_counter = 0

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                # Scale for real-time data displaying
                scale = 0.1
                resized_image = cv2.resize(self.color_value, None, fx=scale, fy=scale, 
                                          interpolation=cv2.INTER_AREA)
                cv2.circle(resized_image, (int(self.width*scale//2), int(self.height*scale//2)), 
                          1, (0, 255, 0))
                display_color.value = bgr8_to_jpeg(resized_image)
                
                self.depth_image = np.asanyarray(self.depth.get_data())

                # Process depth image for curious behavior
                depth_image_test = self.depth_image.copy()
                depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)
                
                depth_image_test[:94, :] = 0
                depth_image_test[282:, :] = 0
                depth_image_test[:, :168] = 0
                depth_image_test[:, 504:] = 0

                # Filter depth values
                depth_image_test[depth_image_test < 100] = 0
                depth_image_test[depth_image_test > 2000] = 0
                
                depth_colormap = cv2.applyColorMap(
                    cv2.convertScaleAbs(depth_image_test, alpha=0.03), 
                    cv2.COLORMAP_JET)
                
                # CURIOUS BEHAVIOR - three-state exploration
                if self.curiosity_state == 'scanning':
                    if depth_image_test.max() == 0:
                        # No object detected - keep searching
                        robot.spinRight(0.3)
                        cv2.putText(depth_colormap, 'SCANNING', (260, 188), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
                        print('Curious: Scanning for objects', end='\r')
                    else:
                        # Object found - switch to approach
                        self.curiosity_state = 'approach'
                        
                elif self.curiosity_state == 'approach':
                    if depth_image_test.max() == 0:
                        # Lost object - return to scanning
                        self.curiosity_state = 'scanning'
                    else:
                        distance = depth_image_test[depth_image_test != 0].min()
                        
                        if distance > 700:
                            # Move toward object
                            robot.forward(0.4)
                            cv2.putText(depth_colormap, f'APPROACH {distance:.0f}mm', (220, 188),
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 165, 0), 2, cv2.LINE_AA)
                            print(f'Curious: Approaching {distance:.0f}mm', end='\r')
                        else:
                            # Close enough - start investigation
                            self.curiosity_state = 'investigate'
                            self.investigation_counter = 0
                            
                elif self.curiosity_state == 'investigate':
                    # Circle around object
                    robot.spinRight(0.4)
                    self.investigation_counter += 1
                    cv2.putText(depth_colormap, f'INVESTIGATE {self.investigation_counter}/60', (200, 188),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (147, 20, 255), 2, cv2.LINE_AA)
                    print(f'Curious: Investigating ({self.investigation_counter}/60)', end='\r')
                    
                    if self.investigation_counter > 60:
                        # Finished investigation - return to scanning
                        self.curiosity_state = 'scanning'
                    
                resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale,
                                                    interpolation=cv2.INTER_AREA)
                display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

**To stop the camera and robot:**

In [ ]:
camera.stop()

The curious behavior demonstrates active exploration through three distinct states:

1. **Scanning State**: Robot rotates slowly (spinRight at 0.3 speed) searching for objects in the environment
2. **Approach State**: When object detected, robot moves forward (0.4 speed) until reaching ~700mm distance
3. **Investigate State**: Robot circles around object (spinRight at 0.4 speed) for 60 frames to examine it from multiple angles

After investigation completes, the robot returns to scanning state to continue exploring. If an object is lost during approach, the robot immediately returns to scanning.